In [1]:
import numpy as np
import pandas as pd

### TODO:
* sklearn text
* spaCy
* transformers
* Anomaly detection in NLP
### 19.04
* Бейзлайн за 2 часа
* Полный pipeline для обучения за 2 часа
* Ресерч техник, нейронки
* Исследовать метрику (чем плоха f1)
####
Умные мысли
* насколько числа играют важную роль?
* играет ли роль стемминг?
* играет ли роль нормализация?

In [2]:
train_df = pd.read_csv("../dataset/train.tsv", sep="\t")
test_df = pd.read_csv("../dataset/train.tsv", sep="\t")

# Preprcessing
## TODO:
* lower()
* remove punkt
* change digits to words

In [3]:
import string
from nltk.corpus import stopwords
from num2words import num2words

def text_process(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    STOPWORDS = stopwords.words('russian')
    ext_punt = string.punctuation + "«" + "»"
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in ext_punt]
    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)

    tokens = []
    for token in nopunc.split(" "):
        value = token
        try:
            value = int(token)
            value = num2words(value, lang='ru')
        except ValueError:
            pass
        finally:
            tokens.append(value)

    nopunc = " ".join(tokens)
    # Now just remove any stopwords
    return ' '.join([word for word in nopunc.split() if word.lower() not in STOPWORDS])

In [4]:
train_df['clean_title'] = train_df['title'].apply(text_process)
# есть ли смысл убирать римские цифры и тд?

In [5]:
train_df['clean_title']

0       Москвичу Владимиру Клутину пришёл счёт вмешате...
1       Агент Кокорина назвал езду встречке житейской ...
2       Госдума рассмотрит возможность введения секрет...
3       ФАС заблокировала поставку скоростных трамваев...
4       Против Навального завели дело недоносительстве...
                              ...                        
5753     Эдди Чемберс получил сотрясение мозга бою Кличко
5754    Правительство застроит Россию нефтепродуктопро...
5755    стыдно дедом новый канцлер ФРГ обратился перво...
5756         Туркмения декабре начнет поставки газа Китай
5757    Бывший тренер Локомотива возглавил нальчикский...
Name: clean_title, Length: 5758, dtype: object

## Adding meta-features


In [27]:
from slovnet import NER
from navec import Navec
navec = Navec.load("../slovnet/navec_news_v1_1B_250K_300d_100q.tar")
ner = NER.load("../slovnet/slovnet_ner_news_v1.tar")
ner.navec(navec)
from collections import Counter
def get_ner(text : str):
    c = Counter()
    for span in ner(text).spans:
        c[span.type] += 1
    return pd.Series(c.values(), index=list(c.keys()), dtype='int')

In [30]:
train_df[['person','organization' , 'location']] = train_df['title'].apply(lambda x: get_ner(x)).fillna(0).astype(int)

,title,is_fake,clean_title,person,organization,location
0,Москвичу Владимиру Клутину пришёл счёт за вмеш...,1,Москвичу Владимиру Клутину пришёл счёт вмешате...,1,0,0
1,Агент Кокорина назвал езду по встречке житейск...,0,Агент Кокорина назвал езду встречке житейской ...,1,0,0
2,Госдума рассмотрит возможность введения секрет...,1,Госдума рассмотрит возможность введения секрет...,0,1,0
3,ФАС заблокировала поставку скоростных трамваев...,0,ФАС заблокировала поставку скоростных трамваев...,0,1,1
4,Против Навального завели дело о недоносительст...,1,Против Навального завели дело недоносительстве...,2,0,0
...,...,...,...,...,...,...
5753,Эдди Чемберс получил сотрясение мозга в бою с ...,0,Эдди Чемберс получил сотрясение мозга бою Кличко,2,0,0
5754,Правительство застроит Россию нефтепродуктопро...,0,Правительство застроит Россию нефтепродуктопро...,0,0,1
5755,«Мне стыдно перед дедом»: новый канцлер ФРГ об...,1,стыдно дедом новый канцлер ФРГ обратился перво...,1,0,1
5756,Туркмения в декабре начнет поставки газа в Китай,0,Туркмения декабре начнет поставки газа Китай,0,0,2


In [31]:
train_df['word_count'] = train_df['clean_title'].apply(lambda x: len(str(x).split()))
# вероятно будут слишком сильно коррелировать поэтому нужно убрать что-то одно
train_df['mean_word_length'] = train_df['clean_title'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
train_df['char_count'] = train_df['title'].apply(lambda x: len(str(x)))

In [32]:
train_df

,title,is_fake,clean_title,person,organization,location,word_count,mean_word_length,char_count
0,Москвичу Владимиру Клутину пришёл счёт за вмеш...,1,Москвичу Владимиру Клутину пришёл счёт вмешате...,1,0,0,8,8.125000,77
1,Агент Кокорина назвал езду по встречке житейск...,0,Агент Кокорина назвал езду встречке житейской ...,1,0,0,7,6.857143,57
2,Госдума рассмотрит возможность введения секрет...,1,Госдума рассмотрит возможность введения секрет...,0,1,0,8,8.500000,75
3,ФАС заблокировала поставку скоростных трамваев...,0,ФАС заблокировала поставку скоростных трамваев...,0,1,1,6,8.000000,57
4,Против Навального завели дело о недоносительст...,1,Против Навального завели дело недоносительстве...,2,0,0,6,8.166667,59
...,...,...,...,...,...,...,...,...,...
5753,Эдди Чемберс получил сотрясение мозга в бою с ...,0,Эдди Чемберс получил сотрясение мозга бою Кличко,2,0,0,7,6.000000,52
5754,Правительство застроит Россию нефтепродуктопро...,0,Правительство застроит Россию нефтепродуктопро...,0,0,1,4,12.250000,52
5755,«Мне стыдно перед дедом»: новый канцлер ФРГ об...,1,стыдно дедом новый канцлер ФРГ обратился перво...,1,0,1,9,6.222222,81
5756,Туркмения в декабре начнет поставки газа в Китай,0,Туркмения декабре начнет поставки газа Китай,0,0,2,6,6.500000,48


# Pipeline

In [54]:
y = train_df['is_fake']
train =  train_df.drop(columns=['is_fake', 'title'])
print(train.shape)
print(y.shape)

(5758, 7)
(5758,)


In [64]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn import model_selection
from sklearn.compose import ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('title_bow', CountVectorizer(), 'clean_title'),
    ], remainder='passthrough', sparse_threshold=0
)
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
pipe1 = Pipeline([('preprocessor', preprocessor),   
                 ('model', MultinomialNB())])
# Vectorizer(Count, tfidf, both), model(Bayes, SVD, LogReg, word2vec*, nn*), * different preprocessing

In [65]:
def check_params_exist(esitmator, params_keyword):
    all_params = esitmator.get_params().keys()
    available_params = [x for x in all_params if params_keyword in x]
    if len(available_params)==0:
        return "No matching params found!"
    else:
        return available_params
print(check_params_exist(pipe1, 'title_bow'))

['preprocessor__title_bow', 'preprocessor__title_bow__analyzer', 'preprocessor__title_bow__binary', 'preprocessor__title_bow__decode_error', 'preprocessor__title_bow__dtype', 'preprocessor__title_bow__encoding', 'preprocessor__title_bow__input', 'preprocessor__title_bow__lowercase', 'preprocessor__title_bow__max_df', 'preprocessor__title_bow__max_features', 'preprocessor__title_bow__min_df', 'preprocessor__title_bow__ngram_range', 'preprocessor__title_bow__preprocessor', 'preprocessor__title_bow__stop_words', 'preprocessor__title_bow__strip_accents', 'preprocessor__title_bow__token_pattern', 'preprocessor__title_bow__tokenizer', 'preprocessor__title_bow__vocabulary']


In [63]:
model_selection.cross_val_score(pipe1, train, y, cv=cv, n_jobs=-1, scoring='f1')

array([0.82053571, 0.81922399, 0.82258782])

In [33]:
model_selection.cross_val_score(pipe2, train_df['clean_title'], train_df['is_fake'], cv=cv, n_jobs=-1, scoring='f1')

array([0.81558442, 0.81515617, 0.81295716])

In [50]:
# perform cross validation with metrics
from sklearn.metrics import classification_report, f1_score, confusion_matrix
def perform_fit(pipe : Pipeline, data : pd.Series, target : pd.Series):
    kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    for fold, (train_ids, valid_ids) in enumerate(kfold.split(data, target)):
        pipe.fit(data.loc[train_ids], target.loc[train_ids])
        y_true = target.loc[valid_ids]
        preds = pipe.predict(data.loc[valid_ids])
        print(f"Classification report for fold - {fold}")
        print("F1 score - ", round(f1_score(y_true, preds), 4))
        print("Conf matrix:")
        print(confusion_matrix(y_true, preds))
        # print(classification_report(, preds, target_names=['not fake', 'fake']))

# Test vectorization
https://andhint.github.io/machine-learning/nlp/Feature-Extraction-From-Text/

In [51]:
pipe = Pipeline([('bow', CountVectorizer()),
                 # ('tfid', TfidfTransformer()),
                 ('model', MultinomialNB())])
perform_fit(pipe, train_df['clean_title'], train_df['is_fake'])

Classification report for fold - 0
F1 score -  0.8289
Conf matrix:
[[746 214]
 [129 831]]
Classification report for fold - 1
F1 score -  0.8265
Conf matrix:
[[742 217]
 [131 829]]
Classification report for fold - 2
F1 score -  0.8287
Conf matrix:
[[743 217]
 [127 832]]


In [59]:
from sklearn.model_selection import GridSearchCV
pipe = Pipeline([('bow', CountVectorizer()),
                 # ('tfid', TfidfTransformer()),
                 ('model', MultinomialNB())])
param_grid = {
    "bow__ngram_range": [(1, 1), (1, 2), (1, 3), (2, 3)],
    "bow__max_df": [0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 0.7, 0.8, 1.0],
    "bow__min_df": [2, 3, 4, 1],
    # "bow__max_features": [2, 3, 4],
}
search = GridSearchCV(pipe, param_grid, cv=cv, scoring='f1' ,n_jobs=-1)
search.fit(train_df['clean_title'], train_df['is_fake'])
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)
# вывод по max_df - очень малое кол-во слов отличает fake от не fake

Best parameter (CV score=0.829):
{'bow__max_df': 0.1, 'bow__min_df': 1, 'bow__ngram_range': (1, 3)}


In [61]:
from sklearn.model_selection import GridSearchCV
pipe = Pipeline([('bow', CountVectorizer()),
                 ('tfid', TfidfTransformer()),
                 ('model', MultinomialNB())])
param_grid = {
    "bow__ngram_range": [(1, 1), (1, 2), (1, 3), (2, 3)],
    "bow__max_df": [0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 0.7, 0.8, 1.0],
    "bow__min_df": [2, 3, 4, 1],
    # "bow__max_features": [2, 3, 4],
}
search = GridSearchCV(pipe, param_grid, cv=cv, scoring='f1' ,n_jobs=-1)
search.fit(train_df['clean_title'], train_df['is_fake'])
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

Best parameter (CV score=0.830):
{'bow__max_df': 0.1, 'bow__min_df': 1, 'bow__ngram_range': (1, 3)}


In [64]:


from IPython.core.display_functions import display

pipe = Pipeline([('bow', CountVectorizer(max_df=0.1, ngram_range=(1, 3))),
                 ('tfid', TfidfTransformer()),
                 ('model', MultinomialNB())])
data = train_df['clean_title']
target = train_df['is_fake']
for fold, (train_ids, valid_ids) in enumerate(cv.split(data, target)):
    pipe.fit(data.loc[train_ids], target.loc[train_ids])
    y_true = target.loc[valid_ids]
    test = data.loc[valid_ids]
    preds = pipe.predict(test)
    print(f"Classification report for fold - {fold}")
    print("F1 score - ", round(f1_score(y_true, preds), 4))
    print("Conf matrix:")
    print(confusion_matrix(y_true, preds))
    print("False positive")
    display(test[preds > y_true])
    print("False negative")
    display(test[preds < y_true])
    break

Classification report for fold - 0
F1 score -  0.8332
Conf matrix:
[[755 205]
 [128 832]]
False positive


14            Режиссера Алексея Германа наградили орденом
35       U2 Metallica сыграют юбилее Зала славы рокнролла
81            Польше готовятся издать поэму Папы Римского
83      Олимпийские винтовки Бьорндалена ошибке отправ...
158     Правительство России дорого заплатит федеральн...
                              ...                        
5708    Шарон Стоун Джо Пеши сыграли самую худшую любо...
5712     Сотрудники МВД пришли документами Росстрахнадзор
5732                Вдова Джорджа Харрисона посвятила сад
5733    Силуанов пересчитал оставшиеся резервах России...
5754    Правительство застроит Россию нефтепродуктопро...
Name: clean_title, Length: 205, dtype: object

False negative


30      Ольгино объяснили большое количество комментар...
75      Минэнерго ФРГ выпустило обучающие ролики пошив...
113                   Domestos разработает свой антивирус
145     известной картине Леонардо Винчи обнаружили QRкод
146     “Дальше Божьей помощью” Москве мужчину пустили...
                              ...                        
5531    Автором экстремистского произведения Протоколы...
5601    приема беженцев Литве восстановлена инфраcтрук...
5611     Великобритании начали разыгрывать бензин лотерею
5742    Газпром Минцифры запускают конкурс самый зрели...
5755    стыдно дедом новый канцлер ФРГ обратился перво...
Name: clean_title, Length: 128, dtype: object

In [67]:
import time
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pickle

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

preprocessor_cv = ColumnTransformer(
    transformers=[
        ('bow', CountVectorizer(), 'clean_title'),
    ], remainder='passthrough', sparse_threshold=0
)
preprocessor_tf = ColumnTransformer(
    transformers=[
        ('bow', TfidfVectorizer(), 'clean_title'),
    ], remainder='passthrough', sparse_threshold=0
)
pipe_bow = [("pre", preprocessor_cv), ("pre", preprocessor_tf)]

bow_param_grid = {
    "pre__bow__ngram_range": [(1, 1), (1, 2), (1, 3)],
    # "bow__ngram_range": [(1, 1), (1, 2), (1, 3), (2, 3)],
    "pre__bow__max_df": [0.1, 0.2, 0.5, 1.0],
    # "bow__max_df": [0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 0.7, 0.8, 1.0],
    # "bow__min_df": [2, 3, 4, 1],
    "pre__bow__min_df": [1],
    # "bow__max_features": [2, 3, 4],
}
pipe_classifier = [('nb', MultinomialNB()), ('lr', LogisticRegression()), ('svc', SVC())]
class_grid = {
    'nb' : {
        "nb__alpha" : [1., 1.2, 1.5, 2., 3., 4.]
    },
    'lr': {
        "lr__penalty" : ['l1', 'l2'],
        "lr__solver" : ['liblinear'],
        # "lr__max_iter" : [100, 200, 300],
        "lr__C" : list(np.logspace(-2, 1, num=10)),
        "lr__random_state" : [42],
    },
    'svc': {
        "svc__C" : list(np.logspace(-2, 1, num=10)),
        "svc__kernel" : ['linear', 'poly', 'sigmoid'],
        "svc__degree" : [2, 3],
    }
}
searches = []
for bow in pipe_bow:
    for cls_ in pipe_classifier:
        pipe = Pipeline([bow, cls_])
        param_grid = {**bow_param_grid , **class_grid[cls_[0]]}
        print(f"Testing pipe - {pipe}")
        print(param_grid)
        search = GridSearchCV(pipe, param_grid, cv=cv, scoring='f1' ,n_jobs=-1)
        start = time.time()
        search.fit(train, y)
        print(f"Best parameter (CV score={search.best_score_:.3f}): +- {np.std(search.cv_results_['mean_test_score'])}")
        print(search.best_params_)
        print("Time - ", time.time() - start)
        searches.append(search)

with open("searches.pkl", 'wb') as f:
    pickle.dump(searches, f)

Testing pipe - Pipeline(steps=[('pre',
                 ColumnTransformer(remainder='passthrough', sparse_threshold=0,
                                   transformers=[('bow', CountVectorizer(),
                                                  'clean_title')])),
                ('nb', MultinomialNB())])
{'pre__bow__ngram_range': [(1, 1), (1, 2), (1, 3)], 'pre__bow__max_df': [0.1, 0.2, 0.5, 1.0], 'pre__bow__min_df': [1], 'nb__alpha': [1.0, 1.2, 1.5, 2.0, 3.0, 4.0]}
Best parameter (CV score=0.821): +- 0.047320427391588975
{'nb__alpha': 1.0, 'pre__bow__max_df': 0.1, 'pre__bow__min_df': 1, 'pre__bow__ngram_range': (1, 1)}
Time -  133.51488876342773
Testing pipe - Pipeline(steps=[('pre',
                 ColumnTransformer(remainder='passthrough', sparse_threshold=0,
                                   transformers=[('bow', CountVectorizer(),
                                                  'clean_title')])),
                ('lr', LogisticRegression())])
{'pre__bow__ngram_range': [(1, 1), 

In [19]:
pipe = Pipeline([("bow", VecTfidfTransformer()), ('lr', LogisticRegression())])
param_grid = {"lr__penalty": ['l1', 'l2'],
 "lr__solver": ['liblinear'],
 # "lr__max_iter" : [100, 200, 300],
 "lr__C": list(np.logspace(-2, 1, num=10)),
 "lr__random_state": [42],
 }
search = GridSearchCV(pipe, param_grid, cv=cv, scoring='f1', n_jobs=-1)
start = time.time()
search.fit(train_df['clean_title'], train_df['is_fake'])
print(f"Best parameter (CV score={search.best_score_:.3f}): +- {np.std(search.cv_results_['mean_test_score'])}")

Best parameter (CV score=nan): +- nan


W:\Work\Anaconda\envs\TORCHGPU\lib\site-packages\sklearn\model_selection\_search.py:969: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan]
  warnings.warn(


In [23]:
print(VecTfidfTransformer().fit_transform(train_df['clean_title']))

  (0, 8615)	1
  (0, 2653)	1
  (0, 6723)	1
  (0, 12805)	1
  (0, 15919)	1
  (0, 2677)	1
  (0, 1113)	1
  (0, 3120)	1
  (1, 921)	1
  (1, 6773)	1
  (1, 8852)	1
  (1, 4946)	1
  (1, 3052)	1
  (1, 5094)	1
  (1, 6219)	1
  (2, 3804)	1
  (2, 13607)	1
  (2, 2809)	1
  (2, 2323)	1
  (2, 14571)	1
  (2, 15632)	1
  (2, 16692)	1
  (2, 6764)	1
  (3, 17173)	1
  (3, 5144)	1
  :	:
  (5754, 12283)	1
  (5754, 14061)	1
  (5754, 5536)	1
  (5754, 9396)	1
  (5755, 17405)	1
  (5755, 6411)	1
  (5755, 9520)	1
  (5755, 9761)	1
  (5755, 10947)	1
  (5755, 15792)	1
  (5755, 4198)	1
  (5755, 13871)	1
  (5755, 2143)	1
  (5756, 3438)	1
  (5756, 9129)	1
  (5756, 6651)	1
  (5756, 12098)	1
  (5756, 4208)	1
  (5756, 16568)	1
  (5757, 2177)	1
  (5757, 16427)	1
  (5757, 2783)	1
  (5757, 15381)	1
  (5757, 7723)	1
  (5757, 8933)	1
